# Prediction of CVSS Scoring

In [1]:
from sklearn.utils import shuffle
import vulns_common

vulns_common.download_nvd_vulns_json()
nvd_vulns = vulns_common.load_nvd_vulns_json('data/nvdcve-1.0*.json')
print('Vulnerability descriptions: ' + str(len(nvd_vulns)))
cpe_names = vulns_common.compile_cpe_names(nvd_vulns)
print('Vulnerable product names:'+str(len(cpe_names)))
data = shuffle(nvd_vulns)#, n_samples=1000)
print('data: ' + str(len(data)))

Vulnerability descriptions: 32963
Vulnerable product names:9176
data: 32963


In [2]:
data[0]

['CVE-2019-11816',
 'Incorrect access control in the WebUI in OPNsense before version 19.1.8, and pfsense before 2.4.4-p3 allows remote authenticated users to escalate privileges to administrator via a specially crafted request.',
 'CWE-284',
 'AV:N/AC:L/Au:S/C:P/I:P/A:P',
 6.5,
 'CVSS:3.0/AV:N/AC:L/PR:H/UI:N/S:U/C:H/I:H/A:H',
 7.2,
 ['netgate', 'pfsense']]

In [3]:
## Classification distribution CVSS2

In [4]:
def update_cvss2_metric_counts(data_):
    cvss2_metrics = ['AV:L/','AV:A/','AV:N/',
                     'AC:L/','AC:M/','AC:H/',
                     'Au:N/','Au:S/','Au:M/',
                     'C:N/','C:P/','C:C/',
                     'I:N/','I:P/','I:C/',
                     '/A:N','/A:P','/A:C']
    metric_counts = dict()
    for m in cvss2_metrics:
        metric_counts[m] = 0

    for d in data_:
        for m in cvss2_metrics:
            if m in d[3]:
                metric_counts[m] = metric_counts[m] + 1
    return metric_counts

In [5]:
update_cvss2_metric_counts(data)

{'AV:L/': 4147,
 'AV:A/': 631,
 'AV:N/': 26634,
 'AC:L/': 18153,
 'AC:M/': 12754,
 'AC:H/': 505,
 'Au:N/': 24929,
 'Au:S/': 6481,
 'Au:M/': 2,
 'C:N/': 10589,
 'C:P/': 16309,
 'C:C/': 4514,
 'I:N/': 10310,
 'I:P/': 16680,
 'I:C/': 4422,
 '/A:N': 13124,
 '/A:P': 12730,
 '/A:C': 5558}

In [6]:
## Classification distribution CVSS3

In [7]:
def update_cvss3_metric_counts(data_):
    cvss3_metrics = ['AV:L/','AV:A/','AV:N/','AV:P/',
                     'AC:L/','AC:H/',
                     'PR:N/','PR:L/','PR:H/',
                     'UI:N/','UI:R/',
                     'S:U/','S:C/',
                     '/C:N/','/C:L/','/C:H/',
                     '/I:N/','/I:L/','/I:H/',
                     '/A:N','/A:L','/A:H']
    metric_counts = dict()
    for m in cvss3_metrics:
        metric_counts[m] = 0

    for d in data_:
        for m in cvss3_metrics:
            if m in d[5]:
                metric_counts[m] = metric_counts[m] + 1
                
    return metric_counts

In [8]:
update_cvss3_metric_counts(data)

{'AV:L/': 5268,
 'AV:A/': 613,
 'AV:N/': 21833,
 'AV:P/': 291,
 'AC:L/': 25704,
 'AC:H/': 2301,
 'PR:N/': 18875,
 'PR:L/': 7453,
 'PR:H/': 1677,
 'UI:N/': 17845,
 'UI:R/': 10160,
 'S:U/': 23171,
 'S:C/': 4834,
 '/C:N/': 5985,
 '/C:L/': 5419,
 '/C:H/': 16601,
 '/I:N/': 8431,
 '/I:L/': 4858,
 '/I:H/': 14716,
 '/A:N': 11158,
 '/A:L': 580,
 '/A:H': 16267}

In [9]:
## classification with classifiers suggested by https://scikit-learn.org/stable/tutorial/machine_learning_map/index.html

In [10]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.pipeline import Pipeline
from nltk.stem import WordNetLemmatizer
from nltk.stem.snowball import SnowballStemmer
from sklearn.naive_bayes import MultinomialNB
from sklearn.linear_model import SGDClassifier
from sklearn.svm import LinearSVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.feature_extraction import text 
from cvsslib import cvss2, cvss3, calculate_vector
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score
import numpy as np
import time
import pandas as pd

class StemmedCountVectorizer(CountVectorizer):
    def build_analyzer(self):
        self.stemmer = SnowballStemmer("english")
        analyzer = super(CountVectorizer, self).build_analyzer()
        return lambda doc: (analyzer(' '.join([self.stemmer.stem(word) for word in doc.split(' ')])))

class StemmedTfidfVectorizer(TfidfVectorizer):
    def build_analyzer(self):
        self.stemmer = SnowballStemmer("english")
        analyzer = super(TfidfVectorizer, self).build_analyzer()
        return lambda doc: (analyzer(' '.join([self.stemmer.stem(word) for word in doc.split(' ')])))

class LemmaCountVectorizer(CountVectorizer):
    def build_analyzer(self):
        self.stemmer = WordNetLemmatizer()
        analyzer = super(CountVectorizer, self).build_analyzer()
        return lambda doc: (analyzer(' '.join([self.stemmer.lemmatize(word) for word in doc.split(' ')])))    

class LemmaTfidfVectorizer(TfidfVectorizer):
    def build_analyzer(self):
        self.stemmer = WordNetLemmatizer()
        analyzer = super(TfidfVectorizer, self).build_analyzer()
        return lambda doc: (analyzer(' '.join([self.stemmer.lemmatize(word) for word in doc.split(' ')])))    

unwanted_words = ['issue','defect','bug','fault','flaw','mistake','error','version','system','because','before','disputed']
stop_words = text.ENGLISH_STOP_WORDS#.union(cpe_names)
stop_words = stop_words.union(unwanted_words)

l = True
ngram_s = 1
ngram_e = 3
df = 1
t = r'(?u)\b\w*[a-zA-Z]{3,}\w*\b'

vectorizers = [
    #CountVectorizer(stop_words=stop_words, lowercase=l, ngram_range=(ngram_s, ngram_e), min_df=df, token_pattern=t),
    #StemmedCountVectorizer(stop_words=stop_words, lowercase=l, ngram_range=(ngram_s, ngram_e), min_df=df, token_pattern=t),
    TfidfVectorizer(stop_words=stop_words, lowercase=l, ngram_range=(ngram_s, ngram_e), min_df=df, token_pattern=t),
    #StemmedTfidfVectorizer(stop_words=stop_words, lowercase=l, ngram_range=(ngram_s, ngram_e), min_df=df, token_pattern=t),
    #LemmaTfidfVectorizer(stop_words=stop_words, lowercase=l, ngram_range=(ngram_s, ngram_e), min_df=df, token_pattern=t),
    #LemmaCountVectorizer(stop_words=stop_words, lowercase=l, ngram_range=(ngram_s, ngram_e), min_df=df, token_pattern=t),
              ]

classifiers = [
               #MultinomialNB(),
               #SGDClassifier(tol=1e-3, shuffle=True),
               LinearSVC(),
               #KNeighborsClassifier()
              ]

#CVSS2
df = pd.DataFrame(data)
df = df.drop(df[df[3] == ''].index)
df = df.drop(df[df[4] == ''].index)
df = df.drop([0,2,5,6,7], axis=1)
df = df.rename(index=str, columns={1: 'text', 3: 'vector'})

print('CVSS2 scoring test. See \'results\' structure after run.')
results = []
for nfold in range(10):
    X_train, X_test, train_cvss_vectors, y_test = train_test_split(df['text'], df['vector'])

    for classifier in classifiers:
        for vectorizer in vectorizers:
            pipe = Pipeline([('vect', vectorizer), ('cls', classifier)])#('scale', StandardScaler(with_mean=False)),
            t = time.time()
            #fit and predict
            pred = pd.DataFrame(X_test)
            for i in range(6):# 6 classes on cvss2 metrics
                vector = train_cvss_vectors.str.split('/').str.get(i)
                pipe.fit(X_train, vector)
                predicted = pipe.predict(X_test)
                pred[i] = predicted
                print('.', end='')

            #calculate cvss2 scores
            for index, value in pred.iterrows():
                vector =''
                for i in range(6):
                    vector = vector + value[i] + '/'
                vector = vector[0:len(vector)-1]
                score = calculate_vector(vector, cvss2)
                pred.at[index, 'score'] = score[0]
                pred.at[index, 'truth'] = df.loc[index][4]

            pred.at[pred['score'] < 3.9, 'severity'] = 'low'
            pred.at[pred['score'] >= 4.0, 'severity'] = 'medium'
            pred.at[pred['score'] >= 7.0, 'severity'] = 'high'
            pred.at[pred['score'] >= 9.0, 'severity'] = 'critical'

            pred.at[pred['truth'] < 3.9, 'severity_truth'] = 'low'
            pred.at[pred['truth'] >= 4.0, 'severity_truth'] = 'medium'
            pred.at[pred['truth'] >= 7.0, 'severity_truth'] = 'high'
            pred.at[pred['truth'] >= 9.0, 'severity_truth'] = 'critical'

            score = f1_score(y_true=pred['severity_truth'], y_pred=pred['severity'], average='micro')
            id = str(classifier)[0:str(classifier).find('(')] + ' ' + str(vectorizer)[0:str(vectorizer).find('(')]
            results.append([id, score, round(time.time() - t, 1)])
            print(' ', end='')
print('done')

C:\Users\ossi.jormakka\AppData\Local\Continuum\anaconda3\lib\site-packages\pandas\core\ops\__init__.py:1115: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  result = method(y)


CVSS2 scoring test. See 'results' structure after run.
...... ...... ...... ...... ...... ...... ...... ...... ...... ...... done


In [11]:
results

[['LinearSVC TfidfVectorizer', 0.8436266395008277, 59.5],
 ['LinearSVC TfidfVectorizer', 0.8408251623583344, 61.4],
 ['LinearSVC TfidfVectorizer', 0.8353495479434611, 54.4],
 ['LinearSVC TfidfVectorizer', 0.8406978224882211, 53.6],
 ['LinearSVC TfidfVectorizer', 0.8395517636572011, 53.4],
 ['LinearSVC TfidfVectorizer', 0.8446453584617345, 53.5],
 ['LinearSVC TfidfVectorizer', 0.8405704826181077, 53.7],
 ['LinearSVC TfidfVectorizer', 0.8413345218387878, 56.1],
 ['LinearSVC TfidfVectorizer', 0.8406978224882211, 60.1],
 ['LinearSVC TfidfVectorizer', 0.8399337832675411, 58.8]]

In [12]:
#CVSS3
df = pd.DataFrame(data)
df = df.drop(df[df[5] == ''].index)
df = df.drop(df[df[6] == ''].index)
df = df.drop([0,2,3,4,7], axis=1)
df = df.rename(index=str, columns={1: 'text', 5: 'vector'})

print('CVSS3 scoring test. See \'results\' structure after run.')
results = []
for nfold in range(10):
    X_train, X_test, train_cvss_vectors, y_test = train_test_split(df['text'], df['vector'])

    for classifier in classifiers:
        for vectorizer in vectorizers:
            pipe = Pipeline([('vect', vectorizer), ('cls', classifier)])
            t = time.time()
            #fit and predict
            pred = pd.DataFrame(X_test)
            for i in range(8):# 8 classes on cvss3 metrics
                vector = train_cvss_vectors.str.split('/').str.get(i+1)
                pipe.fit(X_train, vector)
                predicted = pipe.predict(X_test)
                pred[i] = predicted
                print('.', end='')

            #calculate cvss3 scores
            for index, value in pred.iterrows():
                vector =''
                for i in range(8):
                    vector = vector + value[i] + '/'
                vector = vector[0:len(vector)-1]
                score = calculate_vector(vector, cvss3)
                pred.at[index, 'score'] = score[0]
                pred.at[index, 'truth'] = df.loc[index][6]

            pred.at[pred['score'] < 3.9, 'severity'] = 'low'
            pred.at[pred['score'] >= 4.0, 'severity'] = 'medium'
            pred.at[pred['score'] >= 7.0, 'severity'] = 'high'
            pred.at[pred['score'] >= 9.0, 'severity'] = 'critical'
            pred['severity'] = pred['severity'].values.astype(str)

            pred.at[pred['truth'] < 3.9, 'severity_truth'] = 'low'
            pred.at[pred['truth'] >= 4.0, 'severity_truth'] = 'medium'
            pred.at[pred['truth'] >= 7.0, 'severity_truth'] = 'high'
            pred.at[pred['truth'] >= 9.0, 'severity_truth'] = 'critical'
            pred['severity_truth'] = pred['severity_truth'].values.astype(str)

            score = f1_score(y_true=pred['severity_truth'], y_pred=pred['severity'], average='micro')
            id = str(classifier)[0:str(classifier).find('(')] + ' ' + str(vectorizer)[0:str(vectorizer).find('(')]
            results.append([id, score, round(time.time() - t, 1)])
            results.append(score)
            print(' ', end='')
print('done')

CVSS3 scoring test. See 'results' structure after run.
........ ........ ........ ........ ........ ........ ........ ........ ........ ........ done


In [13]:
results

[['LinearSVC TfidfVectorizer', 0.8174807197943444, 66.1],
 0.8174807197943444,
 ['LinearSVC TfidfVectorizer', 0.8053413310482719, 66.1],
 0.8053413310482719,
 ['LinearSVC TfidfVectorizer', 0.8154812910596972, 65.5],
 0.8154812910596972,
 ['LinearSVC TfidfVectorizer', 0.8116252499285919, 66.1],
 0.8116252499285919,
 ['LinearSVC TfidfVectorizer', 0.8164810054270208, 65.9],
 0.8164810054270208,
 ['LinearSVC TfidfVectorizer', 0.8157669237360754, 65.0],
 0.8157669237360754,
 ['LinearSVC TfidfVectorizer', 0.8106255355612683, 64.2],
 0.8106255355612683,
 ['LinearSVC TfidfVectorizer', 0.8111968009140245, 64.8],
 0.8111968009140245,
 ['LinearSVC TfidfVectorizer', 0.8116252499285919, 64.5],
 0.8116252499285919,
 ['LinearSVC TfidfVectorizer', 0.8094830048557555, 64.8],
 0.8094830048557555]

In [25]:
## CVSS2 resampling

In [14]:
from sklearn.utils import resample
import pandas as pd

def cvss2_sample(data_, min_samples, max_samples):
    cvss2_metrics = [['AV:L/','AV:A/','AV:N/'],
                     ['AC:L/','AC:M/','AC:H/'],
                     ['Au:N/','Au:S/','Au:M/'],
                     ['C:N/','C:P/','C:C/'],
                     ['I:N/','I:P/','I:C/'],
                     ['/A:N','/A:P','/A:C']]

    metric_counts = update_cvss2_metric_counts(data_)
    all_texts = []

    df = pd.DataFrame(data_)
    df = df.drop([0,2,5,6,7],axis=1)
    df = df.drop(df[df[3] == ''].index)
    df = df.drop(df[df[4] == ''].index)
    df = df.rename(index=str, columns={1: 'text', 3: 'vector'})

    for i in range(6):
        texts = pd.DataFrame()
        for m in cvss2_metrics[i]:
            if metric_counts[m] > max_samples:
                resampled = resample(df[df['vector'].str.contains(m)], replace=False, n_samples=max_samples)
                resampled['label'] = m
                texts = pd.concat([texts, resampled], sort=False)
                #print(m+' resampled to '+str(len(resampled)))
            elif metric_counts[m] >= min_samples:
                resampled = pd.DataFrame(df[df['vector'].str.contains(m)])
                resampled['label'] = m
                texts = pd.concat([texts, resampled], sort=False)
        texts = texts.drop(['vector'],axis=1)
        all_texts.append(texts)
        
    return all_texts

cvss2_texts = cvss2_sample(data, 10, 100000)

for t in cvss2_texts:
    print(len(t))

31412
31412
31410
31412
31412
31412


In [15]:
## CVSS3 resampling

In [16]:
def cvss3_sample(data_, min_samples, max_samples):
    cvss3_metrics = [['AV:L/','AV:A/','AV:N/','AV:P/'],
                     ['AC:L/','AC:H/'],
                     ['PR:N/','PR:L/','PR:H/'],
                     ['UI:N/','UI:R/'],
                     ['S:U/','S:C/'],
                     ['/C:N/','/C:L/','/C:H/'],
                     ['/I:N/','/I:L/','/I:H/'],
                     ['/A:N','/A:L','/A:H']]

    metric_counts = update_cvss3_metric_counts(data_)
    all_texts = []

    df = pd.DataFrame(data_)
    df = df.drop([0,2,4,3,7],axis=1)
    df = df.drop(df[df[5] == ''].index)
    df = df.drop(df[df[6] == ''].index)
    df = df.rename(index=str, columns={1: 'text', 5: 'vector'})

    for i in range(8):
        texts = pd.DataFrame()
        for m in cvss3_metrics[i]:
            if metric_counts[m] > max_samples:
                resampled = resample(df[df['vector'].str.contains(m)], replace=False, n_samples=max_samples)
                resampled['label'] = m
                texts = pd.concat([texts, resampled], sort=False)
                #print(m+' resampled to '+str(len(resampled)))
            elif metric_counts[m] >= min_samples:
                resampled = pd.DataFrame(df[df['vector'].str.contains(m)])
                resampled['label'] = m
                texts = pd.concat([texts, resampled], sort=False)
        texts = texts.drop(['vector'],axis=1)
        all_texts.append(texts)
    
    return all_texts

cvss3_texts = cvss3_sample(data, 10, 100000)

for t in cvss3_texts:
    print(len(t))

28005
28005
28005
28005
28005
28005
28005
28005


In [17]:
# Items in vocabulary
# vectorizer and pipeline are defined here
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction import text
from sklearn.svm import LinearSVC
unwanted_words = ['issue','defect','bug','fault','flaw','mistake','error','version','system','because','before','disputed']
stop_words = text.ENGLISH_STOP_WORDS
stop_words = stop_words.union(unwanted_words)
vectorizer = TfidfVectorizer(stop_words=stop_words, ngram_range=(1, 3), token_pattern=r'(?u)\b\w*[a-zA-Z]{3,}\w*\b')
pipe = Pipeline([('vect', vectorizer), ('cls', LinearSVC())])
pipe.fit(cvss3_texts[0]['text'], cvss3_texts[0]['label'])
vectors = vectorizer.transform(cvss3_texts[0]['text'])
print("Vectors shape:" + str(vectors.shape))
vectorizer.vocabulary_

Vectors shape:(28005, 471601)


{'discovered': 127233,
 'sox': 388549,
 'xmalloc': 466361,
 'integer': 216692,
 'overflow': 289171,
 'result': 352839,
 'multiplication': 270684,
 'fed': 162192,
 'lsx_valloc': 248926,
 'macro': 249967,
 'wraps': 464098,
 'malloc': 252972,
 'buffer': 55306,
 'allocated': 19752,
 'smaller': 385095,
 'expected': 155811,
 'leading': 234943,
 'heap': 191567,
 'based': 47667,
 'channels_start': 67909,
 'remix': 344462,
 'discovered sox': 129164,
 'sox xmalloc': 388560,
 'xmalloc integer': 466362,
 'integer overflow': 216716,
 'overflow result': 290243,
 'result multiplication': 353322,
 'multiplication fed': 270689,
 'fed lsx_valloc': 162195,
 'lsx_valloc macro': 248927,
 'macro wraps': 250020,
 'wraps malloc': 464099,
 'malloc buffer': 252975,
 'buffer allocated': 55328,
 'allocated smaller': 19831,
 'smaller expected': 385101,
 'expected leading': 155847,
 'leading heap': 235115,
 'heap based': 191583,
 'based buffer': 47746,
 'buffer overflow': 55551,
 'overflow channels_start': 289365,


In [18]:
## Classifier hyperparameter tuning with GridSearchCV

In [ ]:
# Takes hours with the full dataset
from sklearn.model_selection import GridSearchCV

#hyperparameters
parameters = {'cls__loss': ('hinge', 'squared_hinge'),#default: squared_hinge
              'cls__dual': (True, False),#default: True
              'cls__multi_class': ('ovr', 'crammer_singer'),#default: ovr
              'cls__max_iter': (1000, 2000),#default: 1000
             }
gs = GridSearchCV(pipe, parameters, scoring='f1_micro', cv=10, error_score=np.nan)

for i in range(6):# 6 classes on cvss2 metrics
    gs = gs.fit(cvss2_texts[i]['text'], cvss2_texts[i]['label'])
    print(str(i)+':'+str(gs.best_score_))
    print(str(i)+':'+str(gs.best_params_))

C:\Users\ossi.jormakka\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\svm\base.py:922: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)
C:\Users\ossi.jormakka\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\model_selection\_validation.py:559: FitFailedWarning: Estimator fit failed. The score on this train-test partition for these parameters will be set to nan. Details: 
ValueError: Unsupported set of arguments: The combination of penalty='l2' and loss='hinge' are not supported when dual=False, Parameters: penalty='l2', loss='hinge', dual=False

  FitFailedWarning)
C:\Users\ossi.jormakka\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\model_selection\_search.py:841: DeprecationWarning: The default of the `iid` parameter will change from True to False in version 0.22 and will be removed in 0.24. This will change numeric results when test-set sizes are unequal.
  Dep

0:0.9579857757485293
0:{'cls__dual': True, 'cls__loss': 'squared_hinge', 'cls__max_iter': 1000, 'cls__multi_class': 'ovr'}


C:\Users\ossi.jormakka\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\svm\base.py:922: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)
C:\Users\ossi.jormakka\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\model_selection\_validation.py:559: FitFailedWarning: Estimator fit failed. The score on this train-test partition for these parameters will be set to nan. Details: 
ValueError: Unsupported set of arguments: The combination of penalty='l2' and loss='hinge' are not supported when dual=False, Parameters: penalty='l2', loss='hinge', dual=False

  FitFailedWarning)
C:\Users\ossi.jormakka\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\model_selection\_search.py:841: DeprecationWarning: The default of the `iid` parameter will change from True to False in version 0.22 and will be removed in 0.24. This will change numeric results when test-set sizes are unequal.
  Dep

1:0.9210467385393006
1:{'cls__dual': True, 'cls__loss': 'squared_hinge', 'cls__max_iter': 1000, 'cls__multi_class': 'ovr'}


C:\Users\ossi.jormakka\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\svm\base.py:922: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)
C:\Users\ossi.jormakka\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\model_selection\_validation.py:559: FitFailedWarning: Estimator fit failed. The score on this train-test partition for these parameters will be set to nan. Details: 
ValueError: Unsupported set of arguments: The combination of penalty='l2' and loss='hinge' are not supported when dual=False, Parameters: penalty='l2', loss='hinge', dual=False

  FitFailedWarning)
C:\Users\ossi.jormakka\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\model_selection\_search.py:841: DeprecationWarning: The default of the `iid` parameter will change from True to False in version 0.22 and will be removed in 0.24. This will change numeric results when test-set sizes are unequal.
  Dep

2:0.9225113353212696
2:{'cls__dual': True, 'cls__loss': 'squared_hinge', 'cls__max_iter': 1000, 'cls__multi_class': 'ovr'}


C:\Users\ossi.jormakka\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\svm\base.py:922: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)
C:\Users\ossi.jormakka\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\model_selection\_validation.py:559: FitFailedWarning: Estimator fit failed. The score on this train-test partition for these parameters will be set to nan. Details: 
ValueError: Unsupported set of arguments: The combination of penalty='l2' and loss='hinge' are not supported when dual=False, Parameters: penalty='l2', loss='hinge', dual=False

  FitFailedWarning)
C:\Users\ossi.jormakka\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\model_selection\_search.py:841: DeprecationWarning: The default of the `iid` parameter will change from True to False in version 0.22 and will be removed in 0.24. This will change numeric results when test-set sizes are unequal.
  Dep

3:0.9051317967655673
3:{'cls__dual': True, 'cls__loss': 'squared_hinge', 'cls__max_iter': 1000, 'cls__multi_class': 'ovr'}


C:\Users\ossi.jormakka\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\svm\base.py:922: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)
C:\Users\ossi.jormakka\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\model_selection\_validation.py:559: FitFailedWarning: Estimator fit failed. The score on this train-test partition for these parameters will be set to nan. Details: 
ValueError: Unsupported set of arguments: The combination of penalty='l2' and loss='hinge' are not supported when dual=False, Parameters: penalty='l2', loss='hinge', dual=False

  FitFailedWarning)


In [ ]:
for i in range(8):# 8 classes on cvss3 metrics
    gs = gs.fit(cvss3_texts[i]['text'], cvss3_texts[i]['label'])
    print(str(i)+':'+str(gs.best_score_))
    print(str(i)+':'+str(gs.best_params_))

In [19]:
## Test predicting CVSS of a security related issue

In [20]:
import pandas as pd
import random

reports = pd.read_excel('data/sec_issues_edit.xlsx')
reports = reports.loc[(reports['security']==1)]
reports['security'] = reports['security'].values.astype(bool)
reports['report'] = reports['report'].values.astype(str)
i = random.randint(0, len(reports)-1)
random_report = reports.iloc[i]['report']

In [21]:
from cvsslib import cvss2, cvss3, calculate_vector

print(random_report)

report_metrics = []
for i in range(6):# 6 labels on cvss2 metrics
    pipe.fit(cvss2_texts[i]['text'], cvss2_texts[i]['label'])
    predicted = pipe.predict([random_report])
    report_metrics.append(predicted)

vector = ''
for m in report_metrics:
    vector = vector + str(m[0].replace('/','')) + '/'
vector = vector[0:len(vector)-1]
score = calculate_vector(vector, cvss2)
print('CVSS2: ' + vector + ' = ' + str(score[0]))

report_metrics = []
for i in range(8):# 8 classes on cvss3 metrics
    pipe.fit(cvss3_texts[i]['text'], cvss3_texts[i]['label'])
    predicted = pipe.predict([random_report])
    report_metrics.append(predicted)

vector = ''
for m in report_metrics:
    vector = vector + str(m[0].replace('/','')) + '/'
vector = vector[0:len(vector)-1]
score = calculate_vector(vector, cvss3)
print('CVSS3: ' + vector + ' = ' + str(score[0]))

Issue 20233 : Crash potentially due to resource exhaustion 1 person starred this issue and may be notified of changes.   &nbsp;   sg.develop    MustLive sent a report on security@google.com which is included later. A simplified PoC by Michal is: perl -e '{print &quot;&lt;script&gt;location.hash='\''&quot; &quot;A&quot;x20000000 &quot;'\'';&lt;/script&gt;&quot;}' &gt;testme.html ...and it does not seem to be doing much  other than bogging down the UI by the virtue of dumping 20 megs into the URL bar. However  it did crash my Dev channel Chrome when using this PoC and accessing other tabs during the load. http://www.corp.google.com/~sumitg/no_crawl/chrome/mustlive- locationdos.html MustLive report: &quot;I want to warn you about Denial of Service vulnerability in Google Chrome. At the end of December DoS vulnerability in Mozilla Firefox 3.0.5 was found by Jeremy Brown ( http://websecurity.com.ua/2755/ ). After I checked at 23.12.2008 this vulnerability in different browsers  I found that

In [ ]:
## Find optimal dataset size to fit the classifier

In [ ]:
## CVSS2

In [402]:
from cvsslib import cvss2, cvss3, calculate_vector
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score

sample_sizes = [100, 200, 500, 1000, 2000, 3000, 4000, 5000, 6000, 8000, 10000, 12000, 14000, 16000, 18000]

for size in sample_sizes:
    scores = []
    for nfold in range(10):
        df = pd.DataFrame(data)
        df = df.drop(df[df[3] == ''].index)
        df = df.drop(df[df[4] == ''].index)
        df = df.drop([0,2,5,6,7], axis=1)
        df = df.rename(index=str, columns={1: 'text', 3: 'vector'})
        df = df.sample(n=size)
        X_train, X_test, train_cvss_vectors, y_test = train_test_split(df['text'], df['vector'])

        #fit and predict
        pred = pd.DataFrame(X_test)
        for i in range(6):# 6 classes on cvss2 metrics
            vector = train_cvss_vectors.str.split('/').str.get(i)
            pipe.fit(X_train, vector)
            predicted = pipe.predict(X_test)
            pred[i] = predicted

        #calculate cvss2 scores
        for index, value in pred.iterrows():
            vector =''
            for i in range(6):
                vector = vector + value[i] + '/'
            vector = vector[0:len(vector)-1]
            score = calculate_vector(vector, cvss2)
            pred.at[index, 'score'] = score[0]
            pred.at[index, 'truth'] = df.loc[index][4]

        pred.at[pred['score'] < 3.9, 'severity'] = 'low'
        pred.at[pred['score'] >= 4.0, 'severity'] = 'medium'
        pred.at[pred['score'] >= 7.0, 'severity'] = 'high'
        pred.at[pred['score'] >= 9.0, 'severity'] = 'critical'

        pred.at[pred['truth'] < 3.9, 'severity_truth'] = 'low'
        pred.at[pred['truth'] >= 4.0, 'severity_truth'] = 'medium'
        pred.at[pred['truth'] >= 7.0, 'severity_truth'] = 'high'
        pred.at[pred['truth'] >= 9.0, 'severity_truth'] = 'critical'

        scores.append(f1_score(y_true=pred['severity_truth'], y_pred=pred['severity'], average='micro'))

    print(str(size) + ': ' + str(np.mean(scores)))

C:\Users\ossi.jormakka\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\svm\base.py:922: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)


100: 0.6
200: 0.6599999999999999
500: 0.6623999999999999
1000: 0.6896
2000: 0.7132
3000: 0.7358666666666667
4000: 0.7449
5000: 0.7556799999999999
6000: 0.7619333333333332
8000: 0.7847000000000002
10000: 0.78076
12000: 0.7917
14000: 0.7964857142857142
16000: 0.810225
18000: 0.8053555555555555


In [ ]:
## CVSS3

In [403]:
from cvsslib import cvss2, cvss3, calculate_vector
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score

sample_sizes = [100, 200, 500, 1000, 2000, 3000, 4000, 5000, 6000, 8000, 10000, 12000, 14000, 16000, 17900]

for size in sample_sizes:
    scores = []
    for nfold in range(10):
        df = pd.DataFrame(data)
        df = df.drop(df[df[5] == ''].index)
        df = df.drop(df[df[6] == ''].index)
        df = df.drop([0,2,3,4,7], axis=1)
        df = df.rename(index=str, columns={1: 'text', 5: 'vector'})
        df = df.sample(n=size)
        X_train, X_test, train_cvss_vectors, y_test = train_test_split(df['text'], df['vector'])

        #fit and predict
        pred = pd.DataFrame(X_test)
        for i in range(8):# 8 classes on cvss3 metrics
            vector = train_cvss_vectors.str.split('/').str.get(i+1)
            pipe.fit(X_train, vector)
            predicted = pipe.predict(X_test)
            pred[i] = predicted

        #calculate cvss3 scores
        for index, value in pred.iterrows():
            vector =''
            for i in range(8):
                vector = vector + value[i] + '/'
            vector = vector[0:len(vector)-1]
            score = calculate_vector(vector, cvss3)
            pred.at[index, 'score'] = score[0]
            pred.at[index, 'truth'] = df.loc[index][6]

        pred.at[pred['score'] < 3.9, 'severity'] = 'low'
        pred.at[pred['score'] >= 4.0, 'severity'] = 'medium'
        pred.at[pred['score'] >= 7.0, 'severity'] = 'high'
        pred.at[pred['score'] >= 9.0, 'severity'] = 'critical'
        pred['severity'] = pred['severity'].values.astype(str)

        pred.at[pred['truth'] < 3.9, 'severity_truth'] = 'low'
        pred.at[pred['truth'] >= 4.0, 'severity_truth'] = 'medium'
        pred.at[pred['truth'] >= 7.0, 'severity_truth'] = 'high'
        pred.at[pred['truth'] >= 9.0, 'severity_truth'] = 'critical'
        pred['severity_truth'] = pred['severity_truth'].values.astype(str)

        scores.append(f1_score(y_true=pred['severity_truth'], y_pred=pred['severity'], average='micro'))

    print(str(size) + ': ' + str(np.mean(scores)))

100: 0.44799999999999995
200: 0.514
500: 0.6048
1000: 0.652
2000: 0.6868
3000: 0.7089333333333332
4000: 0.7299999999999999
5000: 0.7386400000000001
6000: 0.7474666666666667
8000: 0.7538
10000: 0.7696799999999999
12000: 0.7678333333333334
14000: 0.7804


KeyboardInterrupt: 

In [22]:
## Test cvss classification against the nvd scores

In [23]:
## CVSS2

In [24]:
from cvsslib import cvss2, cvss3, calculate_vector
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report

df = pd.DataFrame(data)
df = df.drop(df[df[3] == ''].index)
df = df.drop(df[df[4] == ''].index)
df = df.drop([0,2,5,6,7], axis=1)
df = df.rename(index=str, columns={1: 'text', 3: 'vector'})

X_train, X_test, train_cvss_vectors, y_test = train_test_split(df['text'], df['vector'])
print('Training data:' + str(len(X_train)) + ' Testing data:' + str(len(X_test)))

#fit and predict
pred = pd.DataFrame(X_test)
for i in range(6):# 6 classes on cvss2 metrics
    vector = train_cvss_vectors.str.split('/').str.get(i)
    pipe.fit(X_train, vector)
    predicted = pipe.predict(X_test)
    pred[i] = predicted

#calculate cvss2 scores
for index, value in pred.iterrows():
    vector =''
    for i in range(6):
        vector = vector + value[i] + '/'
    vector = vector[0:len(vector)-1]
    score = calculate_vector(vector, cvss2)
    pred.at[index, 'score'] = score[0]
    pred.at[index, 'truth'] = df.loc[index][4]

pred.at[pred['score'] < 3.9, 'severity'] = 'low'
pred.at[pred['score'] >= 4.0, 'severity'] = 'medium'
pred.at[pred['score'] >= 7.0, 'severity'] = 'high'
pred.at[pred['score'] >= 9.0, 'severity'] = 'critical'

pred.at[pred['truth'] < 3.9, 'severity_truth'] = 'low'
pred.at[pred['truth'] >= 4.0, 'severity_truth'] = 'medium'
pred.at[pred['truth'] >= 7.0, 'severity_truth'] = 'high'
pred.at[pred['truth'] >= 9.0, 'severity_truth'] = 'critical'

print(classification_report(y_true=pred['severity_truth'], y_pred=pred['severity']))
print(confusion_matrix(y_true=pred['severity_truth'], y_pred=pred['severity']))#, labels=['critical','high','medium','low'])

Training data:23559 Testing data:7853
              precision    recall  f1-score   support

    critical       0.78      0.79      0.79       665
        high       0.70      0.71      0.70      1324
         low       0.79      0.75      0.77       880
      medium       0.88      0.89      0.88      4984

   micro avg       0.83      0.83      0.83      7853
   macro avg       0.79      0.79      0.79      7853
weighted avg       0.83      0.83      0.83      7853

[[ 528   66    1   70]
 [  59  938   16  311]
 [   1   18  662  199]
 [  86  322  162 4414]]


In [25]:
## CVSS3

In [26]:
df = pd.DataFrame(data)
df = df.drop(df[df[5] == ''].index)
df = df.drop(df[df[6] == ''].index)
df = df.drop([0,2,3,4,7], axis=1)
df = df.rename(index=str, columns={1: 'text', 5: 'vector'})

X_train, X_test, train_cvss_vectors, y_test = train_test_split(df['text'], df['vector'])
print('Training data:' + str(len(X_train)) + ' Testing data:' + str(len(X_test)))

#fit and predict
pred = pd.DataFrame(X_test)
for i in range(8):# 8 classes on cvss3 metrics
    vector = train_cvss_vectors.str.split('/').str.get(i+1)#the first split is the CVSS version
    pipe.fit(X_train, vector)
    predicted = pipe.predict(X_test)
    pred[i] = predicted

#calculate cvss3 scores
for index, value in pred.iterrows():
    vector =''
    for i in range(8):
        vector = vector + value[i] + '/'
    vector = vector[0:len(vector)-1]
    score = calculate_vector(vector, cvss3)
    pred.at[index, 'score'] = score[0]
    pred.at[index, 'truth'] = df.loc[index][6]

pred.at[pred['score'] < 3.9, 'severity'] = 'low'
pred.at[pred['score'] >= 4.0, 'severity'] = 'medium'
pred.at[pred['score'] >= 7.0, 'severity'] = 'high'
pred.at[pred['score'] >= 9.0, 'severity'] = 'critical'
pred['severity'] = pred['severity'].values.astype(str)

pred.at[pred['truth'] < 3.9, 'severity_truth'] = 'low'
pred.at[pred['truth'] >= 4.0, 'severity_truth'] = 'medium'
pred.at[pred['truth'] >= 7.0, 'severity_truth'] = 'high'
pred.at[pred['truth'] >= 9.0, 'severity_truth'] = 'critical'
pred['severity_truth'] = pred['severity_truth'].values.astype(str)

print(classification_report(y_true=pred['severity_truth'], y_pred=pred['severity']))
print(confusion_matrix(y_true=pred['severity_truth'], y_pred=pred['severity']))#, labels=['critical','high','medium','low'])

Training data:21003 Testing data:7002
              precision    recall  f1-score   support

    critical       0.65      0.79      0.71      1046
        high       0.80      0.85      0.82      3172
         low       0.54      0.45      0.49        85
      medium       0.91      0.78      0.84      2696
         nan       0.00      0.00      0.00         3

   micro avg       0.81      0.81      0.81      7002
   macro avg       0.58      0.57      0.57      7002
weighted avg       0.82      0.81      0.81      7002

[[ 829  204    1   12    0]
 [ 321 2688    7  156    0]
 [   6   11   38   30    0]
 [ 117  444   24 2111    0]
 [   0    3    0    0    0]]


C:\Users\ossi.jormakka\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\metrics\classification.py:1143: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


In [27]:
## Check NVD vectors and score integrity

In [28]:
## CVSS2

In [29]:
df = pd.DataFrame(data)
df = df.drop(df[df[3] == ''].index)
df = df.drop(df[df[4] == ''].index)

for index, value in df.iterrows():
    score = calculate_vector(value[3], cvss2)
    df.at[index, 'score'] = score[0]

df.at[df['score'] < 3.9, 'severity'] = 'low'
df.at[df['score'] >= 4.0, 'severity'] = 'medium'
df.at[df['score'] >= 7.0, 'severity'] = 'high'
df.at[df['score'] >= 9.0, 'severity'] = 'critical'
df['severity'] = df['severity'].values.astype(str)

df.at[df[4] < 3.9, 'severity_truth'] = 'low'
df.at[df[4] >= 4.0, 'severity_truth'] = 'medium'
df.at[df[4] >= 7.0, 'severity_truth'] = 'high'
df.at[df[4] >= 9.0, 'severity_truth'] = 'critical'
df['severity_truth'] = df['severity_truth'].values.astype(str)
    
print(classification_report(y_true=df['severity_truth'], y_pred=df['severity']))

              precision    recall  f1-score   support

    critical       1.00      1.00      1.00      2636
        high       1.00      1.00      1.00      5234
         low       1.00      1.00      1.00      3500
      medium       1.00      1.00      1.00     20042

   micro avg       1.00      1.00      1.00     31412
   macro avg       1.00      1.00      1.00     31412
weighted avg       1.00      1.00      1.00     31412



In [30]:
## CVSS3

In [31]:
df = pd.DataFrame(data)
df = df.drop(df[df[5] == ''].index)
df = df.drop(df[df[6] == ''].index)

for index, value in df.iterrows():
    score = calculate_vector(value[5], cvss3)
    df.at[index, 'score'] = score[0]

df.at[df['score'] < 3.9, 'severity'] = 'low'
df.at[df['score'] >= 4.0, 'severity'] = 'medium'
df.at[df['score'] >= 7.0, 'severity'] = 'high'
df.at[df['score'] >= 9.0, 'severity'] = 'critical'
df['severity'] = df['severity'].values.astype(str)

df.at[df[6] < 3.9, 'severity_truth'] = 'low'
df.at[df[6] >= 4.0, 'severity_truth'] = 'medium'
df.at[df[6] >= 7.0, 'severity_truth'] = 'high'
df.at[df[6] >= 9.0, 'severity_truth'] = 'critical'
df['severity_truth'] = df['severity_truth'].values.astype(str)
    
print(classification_report(y_true=df['severity_truth'], y_pred=df['severity']))

              precision    recall  f1-score   support

    critical       1.00      1.00      1.00      4082
        high       1.00      1.00      1.00     12672
         low       1.00      1.00      1.00       311
      medium       1.00      1.00      1.00     10931
         nan       1.00      1.00      1.00         9

   micro avg       1.00      1.00      1.00     28005
   macro avg       1.00      1.00      1.00     28005
weighted avg       1.00      1.00      1.00     28005

